In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
# !git clone https://github.com/archana53/power-law-for-CI.git

In [20]:
# !pip install pytorch-lightning

In [21]:
cd /content/power-law-for-CI

/content/power-law-for-CI


In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import torch
from torch.nn import Linear, CrossEntropyLoss, Conv2d, Dropout
from torch.nn.functional import relu, softmax, tanh
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from data_loading import permute_train_test_data
# from pytorch_lightning import LightningModule, Trainer
# from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [23]:
TRANSFORMS = [
        transforms.Pad(2),
        transforms.ToTensor(),
    ]

dataset_tranforms = transforms.Compose(TRANSFORMS)

In [24]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=dataset_tranforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=dataset_tranforms)

In [25]:
training_permutations , test_permutations = permute_train_test_data(mnist_trainset,mnist_testset)

In [26]:
# class Net1(LightningModule):
class Net1(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net1, self).__init__()
        #FC Layers
        self.fc1 = Linear(input_size, hidden_size)
        self.fc2 = Linear(hidden_size, hidden_size)
        self.fc3 = Linear(hidden_size, hidden_size)
        self.fc4 = Linear(hidden_size, output_size)      

        #DropOut Layers
        self.dropout1 = Dropout(0.2)
        self.dropout2 = Dropout(0.5)
        self.dropout3 = Dropout(0.5)

    def forward(self, data):

        data = data.view(-1, input_size)
        data = relu(self.fc1(data))
        data = self.dropout1(data)

        data = relu(self.fc2(data))
        data = self.dropout2(data)

        data = relu(self.fc3(data))
        data = self.dropout3(data)

        data = self.fc3(data)
        return data

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.001)
        

In [27]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [28]:
input_size = 32*32
hidden_size = 400 
num_classes = 10
num_epochs = 1
batch_size = 100
learning_rate = 0.001

In [30]:
for task in range(0, 10):
  net = Net1(input_size, hidden_size, num_classes).to(device)
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
  loss_func = CrossEntropyLoss()
  train_loss = []
  val_loss = []
  train_loader = torch.utils.data.DataLoader(dataset=training_permutations[task], batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(dataset=test_permutations[task], batch_size=batch_size, shuffle=False)
  for epoch in range(num_epochs):
      trainloss = 0.0
      n_total_steps = len(train_loader)
      net.train()
      for i, (images, labels) in enumerate(train_loader):  
          # print(images.shape)
          images = images.to(device)
          labels = labels.to(device)
          outputs = net(images)
          loss = loss_func(outputs, labels)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step() 
          #accumalting training loss for plotting
          trainloss += loss.item()
      train_loss.append(trainloss/n_total_steps)


      net.eval()
      with torch.no_grad():
          n_correct = 0
          n_samples = 0
          valloss = 0.0
          for images, labels in test_loader:
              images = images.to(device)
              labels = labels.to(device)
              outputs = net(images)
              loss = loss_func(outputs, labels)
              valloss += loss.item()
      val_loss.append(valloss/len(test_loader))    

      if (epoch) % 5 == 0:
              print (f'Task {task}, Epoch [{epoch+1}/{num_epochs}], Train Loss: {trainloss/n_total_steps:.4f} Validation Loss: {valloss/len(test_loader):.4f}')

In [30]:
# plt.figure(figsize=(10,10))

# plt.subplot(2,1,1)
# plt.plot(train_loss, label="Training Loss")
# plt.plot(val_loss, label="Validation Loss")
# plt.legend()

# # plt.subplot(2,1,2)
# # plt.plot(acc, label="Training Accuracy")
# # plt.plot(val_ac, label="Validation Accuracy")
# # plt.legend()

# # plt.savefig("/content/drive/My Drive/"+net.name+".png")

In [15]:
# mnist_model = Net1(input_size, hidden_size, num_classes).to(device)

# # Init DataLoader from MNIST Dataset
# train_loader = torch.utils.data.DataLoader(dataset=training_permutations[0], batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_permutations[0], batch_size=batch_size, shuffle=False)

# # Initialize a trainer
# trainer = Trainer(
#     accelerator="auto",
#     devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
#     max_epochs=num_epochs,
#     callbacks=[TQDMProgressBar(refresh_rate=20), EarlyStopping(monitor='val_loss', patience=5)],
# )

# # Train the model ⚡
# trainer.fit(mnist_model, train_loader)